# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116293e+02     1.467991e+00
 * time: 0.4684908390045166
     1     1.086027e+01     8.944963e-01
 * time: 1.046172857284546
     2    -1.206485e+01     1.037327e+00
 * time: 1.117743968963623
     3    -3.371055e+01     7.155700e-01
 * time: 1.2331008911132812
     4    -4.704302e+01     5.844901e-01
 * time: 1.3220138549804688
     5    -5.680233e+01     2.197233e-01
 * time: 1.418823003768921
     6    -5.968755e+01     1.615510e-01
 * time: 1.4921259880065918
     7    -6.090814e+01     5.494498e-02
 * time: 1.5691359043121338
     8    -6.143932e+01     3.887461e-02
 * time: 1.6325268745422363
     9    -6.176597e+01     3.567509e-02
 * time: 1.6963369846343994
    10    -6.192284e+01     2.414319e-02
 * time: 1.7609758377075195
    11    -6.203477e+01     2.061719e-02
 * time: 1.8251049518585205
    12    -6.209673e+01     1.320041e-02
 * time: 1.9004249572753906
    13    -6.213598e+01     9.957523e-03
 * time: 1.9639918804168

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671072
    AtomicLocal         -18.8557698
    AtomicNonlocal      14.8522659
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485376 
    Xc                  -19.3336822

    total               -62.261666459737